<a href="https://colab.research.google.com/github/govardhan-06/DL-models/blob/main/Text_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Jul  2 08:48:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.0

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
import torch
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
from transformers import pipeline,set_seed,AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_metric,load_from_disk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
model_checkpoint="google/pegasus-newsroom"
tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)
model=AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-newsroom and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
raw_datasets = load_dataset("knkarthick/samsum")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

In [16]:
train_dataset=raw_datasets["train"]
validation_dataset=raw_datasets["validation"]
test_dataset=raw_datasets["test"]

In [55]:
def preprocess_Data(batch):
  input_encoding=tokenizer(batch["dialogue"],max_length = 1024, truncation = True )
  with tokenizer.as_target_tokenizer():
    target_encoding=tokenizer(batch["summary"],max_length = 1024, truncation = True )

  return {
      "input_ids":input_encoding["input_ids"],
      "attention_mask":input_encoding["attention_mask"],
      "labels":target_encoding["input_ids"]
  }

In [63]:
def preprocess_Data(batch):

    # Ensure inputs are lists of strings
    dialogues = batch["dialogue"]
    summaries = batch["summary"]

    try:
        # Encode the input texts
        input_encoding = tokenizer(dialogues, max_length=1024, truncation=True, padding="max_length")

        # Encode the target texts
        target_encoding = tokenizer(text_target=summaries, max_length=1024, truncation=True, padding="max_length")

        return {
            "input_ids": input_encoding["input_ids"],
            "attention_mask": input_encoding["attention_mask"],
            "labels": target_encoding["input_ids"],
        }
    except Exception as e:
      # Check the type of inputs
      print(f"Type of dialogue: {type(batch['dialogue'])}")
      print(f"Type of summary: {type(batch['summary'])}")
      # Print the first few elements for debugging
      print(f"Dialogues: {dialogues}")
      print(f"Summaries: {summaries}")
      raise e

# Apply the preprocessing function
raw_datasets_pt = raw_datasets.map(preprocess_Data, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Type of dialogue: <class 'list'>
Type of summary: <class 'list'>
Dialogues: ["Darren: Hi Buddy! Didn't see you at the gym?! You OK?\nRyan: Yeah, mate. Been working bloody nights, too knackered to do much!\nDarren: Nightmare! You coming Friday?\nRyan: Nah, working again, aren't it!? Management don't give a shit about us lot in the warehouse!\nDarren: Yeah, but it's double time, ain't it! You'll be coining it in! See you next week then!\nRyan: Maybe, I dunno... if I can be arsed! I'll give you a bell. See you, mate.", "Adele: What was the name of the show you told me?\nMartin: Money heist. it's on netflix\nAdele: You liked it?\nMartin: a lot! you'll like it too :) it's in spanish\nAdele: OK, I'll check it out tonight\nMartin: Im watching Ozark right now but after money heist it seems so... i dont know... too american :) \nAdele: house of cards season 6 is coming\nMartin: I know, s5 was great. s6 is going to be odd without spacey\nAdele: What do you mean?\nMartin: he's not in the cast\nAd

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [49]:
from transformers import DataCollatorForSeq2Seq

data_collator=DataCollatorForSeq2Seq(tokenizer,model=model)

In [57]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="pegasus-samsum",
    num_train_epochs=5,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=raw_datasets_pt["train"],
    eval_dataset=raw_datasets_pt["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [66]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                             column_text="article",column_summary="highlights"):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], 10))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], 10))

  for article_batch, target_batch in tqdm(
      zip(article_batches, target_batches), total=len(article_batches)):
      inputs=tokenizer(article_batch,max_length=1024,truncation=True,padding="max_length",return_tensors="pt")
      summaries=model.generate(input_ids=inputs["input_ids"].to(device),
                               attention_mask=inputs["attention_mask"].to(device),
                               length_penalty=0.8,
                               num_beams=8,
                               max_length=128,
                               early_stopping=True)
      decoded_summaries=[tokenizer.decode(s,skip_special_tokens=True,
                                          clean_up_tokenization_spaces=True)
                        for s in summaries]
      decoded_summaries=[d.replace("", " ") for d in decoded_summaries]
      metric.add_batch(predictions=decoded_summaries, references=target_batch)
  #Return Rouge scores
  score=metric.compute()
  return score

In [67]:
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric=load_metric('rouge')

<ipython-input-67-ed7b50da1635>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric=load_metric('rouge')


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
